In [ ]:
import scipy.io
import numpy as np
np.set_printoptions(linewidth=200)
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [ ]:
par_data.shape, len(topology), boundary_conditions.shape

In [ ]:
bc = scipy.io.loadmat('BC.mat')['BC']
particles = np.loadtxt("data_particle.dat")
radii = np.loadtxt("data_radii.dat")
cyl = bc[0,0,:]
wall = bc[0,1,:]
#bc[0] += bc[1]
bc, wall

In [ ]:
def ProjectPointsToCylinder(points,cyl):
    point_to_cyl_origin = points-cyl[:3]
    cyl_axis = cyl[3:6]
    cyl_radius = cyl[6]
    cyl_axis /= np.linalg.norm(cyl_axis)

    axis_projection = point_to_cyl_origin*cyl_axis*cyl_axis
    axis_projection += cyl[:3]
    axis_projection_to_points = points-axis_projection
    axis_projection_to_points_scaled = axis_projection_to_points/np.linalg.norm(axis_projection_to_points,axis=1,keepdims=True)*cyl_radius
    point_on_cyl = axis_projection+axis_projection_to_points_scaled
    return point_on_cyl

In [ ]:
def ProjectPointsToHyperplane(points,plane):
    a = wall[:3]-particles                                               # Vector A from particle P to point W on plane
    b = wall[3:6]                                                        # Vector B: normal vector wall
    b /= np.linalg.norm(b)                                                       
    a1 = a*b*b
    point_on_plane = particles+a1
    
    return point_on_plane

In [ ]:
topology_wall = []
for wall_id,wall in enumerate(bc[0]):
    if wall[-1] == 1:
        point_on_wall = ProjectPointsToCylinder(particles,wall)
    if wall[-1] == 0:
        point_on_wall = ProjectPointsToHyperplane(particles,wall)
    P_W_contact = np.linalg.norm(particles-point_on_wall,axis=1)-radii <= 0
    P_idx = np.argwhere(P_W_contact)
    W_idx = np.ones((P_idx.shape[0],1))*-(wall_id+1)
    top_temp = np.concatenate([P_idx,W_idx],axis=1)
    if wall_id == 0:
        topology_wall = top_temp
    else:
        topology_wall = np.concatenate([topology_wall,top_temp])
topology_wall

In [ ]:
point_on_wall = ProjectPointsToCylinder(particles,cyl)
#point_on_wall = ProjectPointsToHyperplane(particles,wall)


In [ ]:
topology_wall = []
for wid in range(len(bc)):
    a = bc[wid,:3]-point[:3]                                                # Vector A from particle P to point W on plane
    b = bc[wid,3:6]                                                         # Vector B: normal vector wall
    a1 = np.abs(np.sum(a*b))                                                # Vector a1(unit) : Absolute size projection A normal to wall
    if a1 - point[3] <= tol*point[3]:
        topology_wall.append([i+1,-(wid+1)])
topology_wall


In [ ]:
from Encoding import BCEncoding,ConstructTopology
ConstructTopology(particles,)
BCEncoding(particles,)

In [ ]:
from Plotting import Plot3DVectors
#fig = plt.figure()
#ax = fig.add_subplot(projection='3d')
#Plot3DVectors(ax,axis_projection,s)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
vector = point_on_wall
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)
vector = particles
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)

In [ ]:
vector = point_on_wall
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
plt.plot(y,z,'o',color='b')
vector = particles
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
plt.plot(y,z,'o',color='r')